<a href="https://colab.research.google.com/github/rachboyle/mixed_methods_b2b_personas_generation/blob/main/web_analytics_percentage_preprocessing_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Web Analytics Pre-Processing and Sanitization Example 

In [ ]:
#Authenticate to GCP to access survey and web analytics data in BigQuery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


##Iterative Mixed Method Approach to B2B SaaS User Personas Overview
Below is an overview of the various analysis and clustering steps we took for our persona generation research. This particular notebook focuses on the step(s) highlighted in yellow: 

1.   **Survey Data Pre-Processing**: Survey data cleaning and pre-processing
2.   **Survey Clustering**: Initial survey clustering to produce rudimentary personas we could use for interview recruitment
3.   <mark>**Analytcs Data Pre-Processing**: Web analytics data (from [Pendo](https://www.pendo.io) tool) pre-processing, manipulation, and sanitization tool
4.   **Preprocessing and Clustering Iterations**: Four iterations of preprocessing and clustering on the web analytics data; the following iterations listed below contain the clustering algorithm, dimensionality reduction method, and data pre-processing manipulation respectively
    * *KMeans, PCA, Averaged Data*
    * *KMeans, PCA, Percentage Usage Data Per Day*
    * *KMeans, UMAP, Percentage Usage Data Per Day*
    * *HDBScan, UMAP, Percentage Usage Data Per Week*







##Imports

In [ ]:
#Install UMAP (dimensionality reduction) and HDBscan (clustering algorithm)
!pip install umap-learn
!pip install hdbscan

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from IPython.display import display_html
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
plt.rcParams.update({'figure.max_open_warning': 0}) #set so you do not get warnings

from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Dimension reduction and clustering libraries
import umap.umap_ as umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.preprocessing import normalize, StandardScaler

##Data Pre-Processing Overview
We processed the data in three ways: 
1. **Averages of the raw data**: on a per day average, user A spends __# clicks on feature X, __ mins on page X)
2. **Average percentages per day data**: on a per day average, __% of user A's total clicks are spent on feature X, __% of user A's total time is spent on page X)  
3. **Average percentages per week data**: on a per week average, __% of user A's total clicks are spent on feature X, __% of user A's total time is spent on page X)

Below is an example of how this was done for the last pre-processing method, *average percentages per week data*. The queries listed below are the original ones used and hence reference an email column. This is illustratory, showing how we handled visitor_id <> email pairings and duplicates. However, in the actual clustering iterations email will be removed in queries and outputs for anonymity.

###Data Aggregation
Aggregate the data by week by assigning each row a year-week id that can then be used to sum event and time data by week per user. Handle duplicate visitors (with the same email).

In [ ]:
#Counts to check against when de-duping the email <> visitor_ids 
%%bigquery --project uxr-design-us-dev duplicates
SELECT "auth visitors w/ duplicate emails" as type, SUM(cnt) as num
FROM (
    SELECT distinct email, COUNT(distinct visitor_id) as cnt 
    FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data`
    WHERE email is not NULL
    GROUP BY email
    ORDER BY cnt DESC
)
WHERE cnt > 1

UNION ALL

SELECT "auth visitors with 1 id" as type, count(email) as num
FROM (
    SELECT distinct email, COUNT(distinct visitor_id) as cnt 
    FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data`
    WHERE email is not NULL
    GROUP BY email
    ORDER BY cnt DESC
)
WHERE cnt = 1

UNION ALL

SELECT "authenticated visitors" as type, count(distinct visitor_id) as num
FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data`
WHERE email is not NULL

UNION ALL

SELECT "anonymous visitors" as type, count(distinct visitor_id) as num
FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data`
WHERE email is NULL

UNION ALL

SELECT "total visitors counting distinct ids" as type, count(*) as num
FROM (
    SELECT distinct visitor_id
    FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data`
)

UNION ALL

SELECT "total visitors counting distinct email <> id" as type, count(*) as num
FROM (
    SELECT distinct email, visitor_id
    FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data`
)

UNION ALL

SELECT "visitor ids with multiple emails" as type, count(visitor_id) as num
FROM (
    SELECT distinct visitor_id, count(distinct email) as cnt
    FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data`
    GROUP BY visitor_id
)
WHERE cnt > 1

In [ ]:
#After investigation, weird edge cases are due to internal testing accounts (i.e. visitor id wth multiple emails)
duplicates

,type,num
0,anonymous visitors,11085
1,total visitors counting distinct ids,34875
2,visitor ids with multiple emails,1
3,authenticated visitors,23790
4,auth visitors w/ duplicate emails,809
5,auth visitors with 1 id,22982
6,total visitors counting distinct email <> id,34876


In [ ]:
#Convert Telemetry Data to be Per Week rather than Per Day, consolidate ids
%%bigquery --project uxr-design-us-dev features_df
with dup_week AS (
  SELECT *, CONCAT(EXTRACT(YEAR from day), "-", EXTRACT(WEEK from day)) as week_num
  FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data` 
  JOIN (
    SELECT email as joined_email, MAX(visitor_id) as true_id FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data`
    WHERE email is not NULL
    GROUP BY joined_email
  )
  ON email = joined_email
),

nondup_week AS (
  SELECT *, CONCAT(EXTRACT(YEAR from day), "-", EXTRACT(WEEK from day)) as week_num
  FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data`
  WHERE email is NULL
)

SELECT app, visitor_id, email, feature_id, feature_name, page_id, page_name,
group_name, is_core_event, week_num, SUM(total_num_events) as events_per_week, SUM(total_num_min) as mins_per_week
FROM nondup_week
GROUP BY app, visitor_id, email, feature_id, feature_name, page_id, page_name, group_name, is_core_event, week_num

UNION ALL

SELECT app, true_id as visitor_id, email, feature_id, feature_name, page_id, page_name,
group_name, is_core_event, week_num, SUM(total_num_events) as events_per_week, SUM(total_num_min) as mins_per_week
FROM dup_week
GROUP BY app, visitor_id, email, feature_id, feature_name, page_id, page_name, group_name, is_core_event, week_num

In [ ]:
#Check the deduping worked as expected so that the deduped data = anonymous users + authenticated users from the original data
%%bigquery --project uxr-design-us-dev deduped_data
SELECT "anonymous users" as label, COUNT(distinct visitor_id) as num
FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data` 
WHERE email is NULL 

UNION ALL 

SELECT "authenticated users" as label, COUNT(distinct visitor_id) as num
FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data` 
WHERE email is not NULL

UNION ALL 

SELECT "deduped data" as label, COUNT(*) as num
FROM (
    SELECT distinct email, visitor_id
    FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data_per_week_deduped` 
)

In [ ]:
deduped_data

,label,num
0,anonymous users,11085
1,authenticated users,23307
2,deduped data,34392


In [ ]:
#Convert Telemetry Data to be Per Week rather than Per Day, consolidate ids
%%bigquery --project uxr-design-us-dev pages_df
with dup_week AS (
  SELECT *, CONCAT(EXTRACT(YEAR from day), "-", EXTRACT(WEEK from day)) as week_num
  FROM `uxr-design-us-dev.clustering_datasets.all_pages_telemetry_data` 
  JOIN (
    SELECT email as joined_email, MAX(visitor_id) as true_id FROM `uxr-design-us-dev.clustering_datasets.all_pages_telemetry_data`
    WHERE email is not NULL
    GROUP BY joined_email
  )
  ON email = joined_email
),

nondup_week AS (
  SELECT *, CONCAT(EXTRACT(YEAR from day), "-", EXTRACT(WEEK from day)) as week_num
  FROM `uxr-design-us-dev.clustering_datasets.all_pages_telemetry_data`
  WHERE email is NULL
)

SELECT app, visitor_id, email, page_id, page_name,
group_name, week_num, SUM(total_num_events) as events_per_week, SUM(total_num_min) as mins_per_week
FROM nondup_week
GROUP BY app, visitor_id, email, page_id, page_name, group_name, week_num

UNION ALL

SELECT app, true_id as visitor_id, email, page_id, page_name,
group_name, week_num, SUM(total_num_events) as events_per_week, SUM(total_num_min) as mins_per_week
FROM dup_week
GROUP BY app, visitor_id, email, page_id, page_name, group_name, week_num

In [ ]:
#Save datasets to bigquery for easy reference
features_df.to_gbq(
    'clustering_datasets.all_features_telemetry_data_per_week_deduped',
    'uxr-design-us-dev', 
     chunksize=None,
     if_exists='replace')

pages_df.to_gbq(
    'clustering_datasets.all_pages_telemetry_data_per_week_deduped',
    'uxr-design-us-dev', 
     chunksize=None,
     if_exists='replace')

###Data Conversion
Convert the average per week data into percentage per week data.

In [ ]:
#Create Per Week Percentage Data for Time on Page
%%bigquery --project uxr-design-us-dev page_df
WITH total_time_per_week AS (
  SELECT visitor_id, week_num, SUM(mins_per_week) as total_mins_per_week
  #dataset w/ page interactions averaged per week
  FROM `uxr-design-us-dev.clustering_datasets.all_pages_telemetry_data_per_week_deduped`
  GROUP BY visitor_id, week_num 
)

SELECT app, d.visitor_id, t.email, page_id, page_name, t.week_num, mins_per_week as page_mins_per_week, total_mins_per_week,
IF(total_mins_per_week = 0, 0, ROUND(mins_per_week / total_mins_per_week,3) * 100) as percent_of_time
FROM `uxr-design-us-dev.clustering_datasets.all_pages_telemetry_data_per_week_deduped`  t
JOIN total_time_per_week d ON d.visitor_id = t.visitor_id AND d.week_num = t.week_num

In [ ]:
#Create Per Week Percentage Data for Feature Clicks
%%bigquery --project uxr-design-us-dev feature_df
WITH total_time_per_week AS (
  SELECT visitor_id, week_num, SUM(events_per_week) as total_events_per_week
  #dataset w/ feature interactions averaged per week
  FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data_per_week_deduped` 
  GROUP BY visitor_id, week_num 
)


SELECT app, d.visitor_id, t.email, feature_id, feature_name, page_id, page_name, t.week_num, events_per_week as feature_events_per_week, total_events_per_week,
IF(total_events_per_week = 0, 0, ROUND(events_per_week/ total_events_per_week,3) * 100) as percent_of_events
FROM `uxr-design-us-dev.clustering_datasets.all_features_telemetry_data_per_week_deduped`  t
JOIN total_time_per_week d ON d.visitor_id = t.visitor_id AND d.week_num = t.week_num

In [ ]:
#Create Average Percentage Data for Feature Clicks
%%bigquery --project uxr-design-us-dev ave_feature_df
SELECT visitor_id, email, feature_id, feature_name, page_id, page_name, group_name, 
AVG(feature_events_per_week) as ave_feature_clicks_per_week, AVG(percent_of_events) as ave_percent_of_events_per_week
FROM `uxr-design-us-dev.mixed_methods_clustering.feature_percentage_data`
GROUP BY visitor_id, email, feature_id, feature_name, page_id, page_name, group_name

In [ ]:
#Create Average Total Percentage Data for Time on Page
%%bigquery --project uxr-design-us-dev ave_page_df
SELECT visitor_id, email, page_id, page_name, group_name, 
AVG(page_mins_per_week) as ave_page_mins_per_week, AVG(percent_of_time) as ave_percent_of_time_per_week
FROM `uxr-design-us-dev.mixed_methods_clustering.page_percentage_data`
GROUP BY visitor_id, email, page_id, page_name, group_name

###Data Sanitization
Sanitize the data so it only users that meet the following qualifications:
* has logged into the platform in the past year
* has been active for the past 10 days
* has at least 30 days of activity data
* is an external user




In [ ]:
print("Number of distinct users in each dataset before sanitization")
unique_feature_users = ave_feature_df.drop_duplicates("visitor_id")
unique_page_users = ave_page_df.drop_duplicates("visitor_id")

print("feature users: %d" % (len(unique_feature_users)))
print("page users   : %d" % (len(unique_page_users)))

Number of distinct users in each dataset before sanitization
feature users: 34392
page users   : 808423


In [ ]:
%%bigquery --project uxr-design-us-dev page_users
SELECT email, MAX(visitor_id) as visitor_id, MIN(day) as first_login, MAX(day) as most_recent_login, COUNT(distinct day) as total_days_active
FROM `clustering_datasets.all_pages_telemetry_data`
WHERE email is not NULL
GROUP BY email

UNION ALL

SELECT email, visitor_id, MIN(day) as first_login, MAX(day) as most_recent_login, COUNT(distinct day) as total_days_active, 
FROM `clustering_datasets.all_pages_telemetry_data`
WHERE email is NULL
GROUP BY email, visitor_id

In [ ]:
total_unique_users = page_users.drop_duplicates("visitor_id")
p_thirty_days_of_activity = page_users["most_recent_login"] - page_users["first_login"] > "30 days"
p_active_for_10_days = page_users["total_days_active"] >= 10
p_login_in_past_year = page_users["most_recent_login"] > "2020-09-15"
external_users = page_users["email"].str.contains("liveramp") == False
active_users = page_users[p_thirty_days_of_activity & p_active_for_10_days & p_login_in_past_year & external_users][["visitor_id", "email"]]

print("\nNumber of distinct users in each dataset after sanitization")
unique_feature_users = sanitized_ave_feature_df.drop_duplicates("visitor_id")
unique_page_users = sanitized_ave_page_df.drop_duplicates("visitor_id")

print("feature users: %d" % (len(unique_feature_users)))
print("page users   : %d" % (len(unique_page_users)))

Total users in dataset: 808423
Total active users: 6342

Number of distinct users in each dataset after sanitization
feature users: 6287
page users   : 6342


In [ ]:
print("Total users in dataset: %d\nTotal active users: %d" % (len(total_unique_users), len(active_users)))
sanitized_ave_page_df = ave_page_df[ave_page_df["visitor_id"].isin(active_users["visitor_id"])]
sanitized_ave_feature_df = ave_feature_df[ave_feature_df["visitor_id"].isin(active_users["visitor_id"])]

###Data Consolidation and Transposition
Transpose and consolidate the data (where each row is a unique pairing of user <> feature / page <> interaction data) to be in a form suitable for clustering by making each row a datapoint (i.e. user <> all interaction data) and each column a clustering feature (i.e. page X, feature Y). 

In [ ]:
#Identify relevant feature ids to use as features in clustering
rel_feature_ids = ["tkt5orHUpCVLIts2gorwT3UgG5I", "Ras6hyvLrUgJiRUgon4y1F4QkN4", "zdpPTMNp1EkXtBZR5v_ObuE7hwA", "C3ryON9ItgJ6UgOwQAZmIE8xa3A", "k39G0EasOmyrjXL-x4x8opM1toI", "m3O0KeqVvHCNvjUgwnA1gphjwBM", "3w9IhLaFCwHOywyY2kj6p7kk21Y", "H_1-rGmIvRDj8hnvXi1xwADN5Z4", "V1XJLd-R1gdR0eiF5Y7bThnGjHw", #advancedtv
                   "GID55RO4sNhBCURo7Z1_lY9NLXs", "HbEegssf9k60Mv8lnrdv_889PHc", "IvuCbXKxt9FCXyAxrnqwFhXPFUM", "E_eF9X73IX3ARmERAfLkPPcHMb4", "56iulnmaAqdLQOD-wy-EtdOMLxw", "UHNiLRNy5-VWEPHlDcq1hAr1DTU", "HBzif5IJ9m2SMxfU1JYbpz5hruo", "VwQHIEgwiQ88mTKFEiIfk4a8aCk", #Data Marketplace
                   "dGro_W0obaKWZs8l-xObcxKmhRg", "GHdt7hBJpluSyeNAQhuekke04S8", "9e_eJvJ3wbmYQlxEI7eadlGc0wA", "CUEn5DTA7EtIh2GfrV6n85e-ugg", "zyt-c4I5PLjo6We_foZiyC0uU8M", "iPN-NrvMGz0wIPCMZV9Vm2CcPyU", "uDfqoLTn6mYspzPtPbKZA4wRtTk", "LQERdzPDKTwyLqEqUOhRLxc2n_Q", "3JO1sgDixw7nC385pm1yyWWHhWw", "vkn0gs9bYzBqFjiHc9wDy5FJbE4", "myTn0VuzM9FtUyijGCGP6Z-vTiI" #Connect
                   ]
print("Number of relevant features: %d\n" % len(rel_feature_ids))

#Original number of features and rows of data
f = len(sanitized_ave_feature_df.drop_duplicates("feature_id"))
r = len(sanitized_ave_feature_df)

sanitized_ave_feature_df = sanitized_ave_feature_df[sanitized_ave_feature_df["feature_id"].isin(rel_feature_ids)]

#Check at the difference in the number of rows and number of features before and after selecting those of interest
print("ALL FEATURES")
print("rows of data: %d\nnumber of features: %d" % (r, f))
print("\nREL FEATURES")
print("rows of data: %d\nnumber of features: %d" % (len(sanitized_ave_feature_df), len(sanitized_ave_feature_df.drop_duplicates("feature_id"))))

Number of relevant features: 28

ALL FEATURES
rows of data: 253605
number of features: 423

REL FEATURES
rows of data: 19248
number of features: 28


In [ ]:
#Identify relevant page ids
admin_removals = ["SideCar - Mapping", "Audiences", "Comopany Settings (All Tabs)", "Fast Lane Sources", "Customer Users", "Company Settings (All Tabs)"]
admin_group_condition = (sanitized_ave_page_df["group_name"] == 'Admin & User Settings') & ~sanitized_ave_page_df["page_name"].isin(admin_removals)

dis_keep = ["Your Destination Accounts", "Your Destination Accounts - My Segments", "Your Destination Accounts - Edit Account", 
            "Your Destination Accounts - My Segments - Delivery Info", "Destinations List", "Choose an Integration Group", "Configure DA Properties" ]
dis_group_condition = (sanitized_ave_page_df["group_name"] == 'Distribution') & sanitized_ave_page_df["page_name"].isin(dis_keep)

dm_removals = ["Data Seller - Upload New", "Data Seller - Upload Existing", "Sell Data (All Tabs)"] #This Sell Data (All Tabs) might be AdvTv
dm_group_condition = (sanitized_ave_page_df["group_name"] == 'Data Marketplace') & ~sanitized_ave_page_df["page_name"].isin(dm_removals)

ing_group_condition = (sanitized_ave_page_df["group_name"] == 'Ingestion & Dashboard') & (sanitized_ave_page_df["page_id"] != "bvIuD4FYA6XkH8iev5cRJyiT5qo") #duplicate page

manage_removals = ["Audience - Users", "Audience - Distribution", "Audience - Google Tab", "Audience Overview", "Audience - Imports"] #extra pages thatt aren't in pendo rn
manage_group_condition = (sanitized_ave_page_df["group_name"] == 'Manage Segments') & (~sanitized_ave_page_df["page_name"].isin(manage_removals))

adv_tv_keep = ['Activity Report', 'Universes (TV Destination)', 'Taxonomy','Audiences', 'Data Sets', 'Builder', 'OTT Hub']
adv_tv_condition = (sanitized_ave_page_df["group_name"].isin(adv_tv_keep)) & (sanitized_ave_page_df["page_name"] != "OTT Hub - Welcome Page")

#Use all conditions together to create the paired down table
final_condition = (adv_tv_condition) | (manage_group_condition) | (ing_group_condition) | (dm_group_condition) | (dis_group_condition) | (admin_group_condition)
rel_page_ids = sanitized_ave_page_df[final_condition]["page_id"].drop_duplicates().to_list()

print("Number of relevant pages: %d\n" % len(rel_page_ids))

#Original number of pages and rows of data
f = len(sanitized_ave_page_df.drop_duplicates("page_id"))
r = len(sanitized_ave_page_df)

sanitized_ave_page_df = sanitized_ave_page_df[sanitized_ave_page_df["page_id"].isin(rel_page_ids)]

#Check at the difference in the number of rows and number of pages before and after selecting those of interest
print("ALL PAGES")
print("rows of data: %d\nnumber of pages: %d" % (r, f))
print("\nREL PAGES")
print("rows of data: %d\nnumber of pages: %d" % (len(sanitized_ave_page_df), len(sanitized_ave_page_df.drop_duplicates("page_id"))))

Number of relevant pages: 52

ALL PAGES
rows of data: 155099
number of pages: 185

REL PAGES
rows of data: 89475
number of pages: 52


In [ ]:
#Transpose the Dataset

#Create dummy tables to feed the transposed dataset  
f_cols = ["visitor_id", "email"] + rel_feature_ids
feature_clustering_table = pd.DataFrame([], columns=f_cols)

p_cols = ["visitor_id", "email"] + rel_page_ids
page_clustering_table = pd.DataFrame([], columns=p_cols)

for index, row in active_users.iterrows():
  #FEATURE DATA
  holder1 = pd.DataFrame([], columns=f_cols)
  visitor_id, email = row

  #Create the feature table by grabbing the minute data for each user
  feature_visitor_data = sanitized_ave_feature_df[sanitized_ave_feature_df["visitor_id"] == visitor_id][["feature_id", "ave_percent_of_events_per_week"]].transpose()

  #Make each distinct feature a column and transpose the data
  feature_visitor_data.columns = feature_visitor_data.iloc[0]
  feature_visitor_data = feature_visitor_data.drop(feature_visitor_data.index[0])

  #Add the visitor and email identifiers to the table
  feature_visitor_data.insert(0, "email", email)
  feature_visitor_data.insert(0, "visitor_id", visitor_id)

  # Add the time data to the page table
  holder1[feature_visitor_data.columns] = feature_visitor_data.head(1)[feature_visitor_data.columns]
  feature_clustering_table = feature_clustering_table.append(holder1, ignore_index=True)

  #PAGE DATA
  holder1 = pd.DataFrame([], columns=p_cols)

  #Create the page table by grabbing the minute data for each user
  page_visitor_data = sanitized_ave_page_df[sanitized_ave_page_df["visitor_id"] == visitor_id][["page_id", "ave_percent_of_time_per_week"]].transpose()

  #Make each distinct feature a column and transpose the data
  page_visitor_data.columns = page_visitor_data.iloc[0]
  page_visitor_data = page_visitor_data.drop(page_visitor_data.index[0])

  #Add the visitor and email identifiers to the table
  page_visitor_data.insert(0, "email", email)
  page_visitor_data.insert(0, "visitor_id", visitor_id)

  # Add the time data to the page table
  holder1[page_visitor_data.columns] = page_visitor_data.head(1)[page_visitor_data.columns]
  page_clustering_table = page_clustering_table.append(holder1, ignore_index=True)

In [ ]:
#Before saving to BQ, reformat the NaNs to 0s and the column headers to be compatible
def reformat(df):
  df = df.fillna(0)
  df[df.columns.tolist()[2::]] = df[df.columns.tolist()[2::]].astype(float) #Takes a while
  df.columns = "_" + df.columns.str.replace('-', '_')
  return df

f_df_reformatted = reformat(feature_clustering_table)
p_df_reformatted = reformat(page_clustering_table)

In [ ]:
f_df_reformatted.to_gbq(
    'mixed_methods_clustering.ave_percentage_of_clicks_per_feature_clustering_dataset',
    'uxr-design-us-dev', 
     chunksize=None, # I have tried with several chunk sizes, it runs faster when it's one big chunk (at least for me)
     if_exists='replace')

p_df_reformatted.to_gbq(
    'mixed_methods_clustering.ave_percentage_of_time_on_page_clustering_dataset',
    'uxr-design-us-dev', 
     chunksize=None, # I have tried with several chunk sizes, it runs faster when it's one big chunk (at least for me)
     if_exists='replace')


1it [00:03,  3.40s/it]
1it [00:05,  5.55s/it]
